# Sentimnet Analyst Classification
##### `Sentiment atau emosi yang diungkapkan dalam teks bersifat positif, negatif dan netral`

# Data Cleansing & Preprocessing

### Mengimpor Data TSV
##### Struktur Data TSV File TSV `terdiri dari beberapa baris` di mana setiap baris merepresentasikan satu `record atau entri data`

In [153]:
import pandas as pd
import numpy as np
import re
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import random

In [154]:
# Memastikan hasil dapat direproduksi
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(27)

In [155]:
import pandas as pd

# Membaca file TSV
df_train = pd.read_csv('train_preprocess.tsv', sep='\t')
df_valid = pd.read_csv('valid_preprocess.tsv', sep='\t')
df_test = pd.read_csv('test_preprocess.tsv', sep='\t')


In [156]:
df_train

,"warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !",positive
0,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
1,lokasi strategis di jalan sumatera bandung . t...,positive
2,betapa bahagia nya diri ini saat unboxing pake...,positive
3,duh . jadi mahasiswa jangan sombong dong . kas...,negative
4,"makanan beragam , harga makanan di food stall ...",positive
...,...,...
10994,tidak kecewa,positive
10995,enak rasa masakan nya apalagi kepiting yang me...,positive
10996,hormati partai-partai yang telah berkoalisi,neutral
10997,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [157]:
df_test

,"kemarin gue datang ke tempat makan baru yang ada di dago atas . gue kira makanan nya enak karena harga nya mahal . ternyata , boro-boro . tidak mau lagi deh ke tempat itu . sudah mana tempat nya juga tidak nyaman banget , terlalu sempit .",negative
0,kayak nya sih gue tidak akan mau balik lagi ke...,negative
1,"kalau dipikir-pikir , sebenarnya tidak ada yan...",negative
2,ini pertama kalinya gua ke bank buat ngurusin ...,negative
3,waktu sampai dengan gue pernah disuruh ibu lat...,negative
4,pelayanan di hotel salak bogor tidak sebagus y...,negative
...,...,...
494,kata nya tidur yang baik itu minimal enam jam ...,neutral
495,indonesia itu ada di benua asia .,neutral
496,salah satu kegemaran anak remaja indonesia sek...,neutral
497,melihat warna hijau bisa bikin mata jadi lebih...,positive


In [158]:
df_valid

,"meski masa kampanye sudah selesai , bukan berati habis pula upaya mengerek tingkat kedipilihan elektabilitas .",neutral
0,tidak enak,negative
1,restoran ini menawarkan makanan sunda . kami m...,positive
2,lokasi di alun alun masakan padang ini cukup t...,positive
3,betapa bejad kader gerindra yang anggota dprd ...,negative
4,kampiun bistro berada di jalan . kebon kawung ...,positive
...,...,...
1254,"film tncfu , tidak cocok untuk penonton yang t...",negative
1255,"indihome ini mahal loh bayar nya . hanya , pen...",negative
1256,"be de gea , cowok cupu yang takut dengan pacar...",negative
1257,valen yang sangat tidak berkualitas . konentat...,negative


### Menambahkan Header pada dataset 

In [159]:
# Membaca file TSV tanpa header terlebih dahulu
df_train = pd.read_csv('train_preprocess.tsv', sep='\t', header=None)
df_valid = pd.read_csv('valid_preprocess.tsv', sep='\t', header=None)
df_test = pd.read_csv('test_preprocess.tsv', sep='\t', header=None)

# Menambahkan header pada DataFrame
df_train.columns = ['review_text', 'category']
df_valid.columns = ['review_text', 'category']
df_test.columns = ['review_text', 'category']

In [160]:
df_train

,review_text,category
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [161]:
df_test

,review_text,category
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative
...,...,...
495,kata nya tidur yang baik itu minimal enam jam ...,neutral
496,indonesia itu ada di benua asia .,neutral
497,salah satu kegemaran anak remaja indonesia sek...,neutral
498,melihat warna hijau bisa bikin mata jadi lebih...,positive


In [162]:
df_valid

,review_text,category
0,"meski masa kampanye sudah selesai , bukan bera...",neutral
1,tidak enak,negative
2,restoran ini menawarkan makanan sunda . kami m...,positive
3,lokasi di alun alun masakan padang ini cukup t...,positive
4,betapa bejad kader gerindra yang anggota dprd ...,negative
...,...,...
1255,"film tncfu , tidak cocok untuk penonton yang t...",negative
1256,"indihome ini mahal loh bayar nya . hanya , pen...",negative
1257,"be de gea , cowok cupu yang takut dengan pacar...",negative
1258,valen yang sangat tidak berkualitas . konentat...,negative


###  Menghitung jumlah setiap label

In [163]:
df_train['category'].value_counts()

category
positive    6416
negative    3436
neutral     1148
Name: count, dtype: int64

In [164]:
df_valid['category'].value_counts()


category
positive    735
negative    394
neutral     131
Name: count, dtype: int64

In [165]:
df_test['category'].value_counts()

category
positive    208
negative    204
neutral      88
Name: count, dtype: int64

In [166]:
import plotly.express as px

# Menghitung jumlah setiap kategori
train_counts = df_train['category'].value_counts().reset_index()
train_counts.columns = ['category', 'count']

valid_counts = df_valid['category'].value_counts().reset_index()
valid_counts.columns = ['category', 'count']

test_counts = df_test['category'].value_counts().reset_index()
test_counts.columns = ['category', 'count']

# Visualisasi distribusi label untuk training set
fig_train = px.bar(train_counts,
                   x='category', y='count',
                   labels={'category':'Kategori', 'count':'Jumlah'},
                   title='Distribusi Label di Training Set',
                   color='category')

fig_train.show()

# Visualisasi distribusi label untuk validation set
fig_valid = px.bar(valid_counts,
                   x='category', y='count',
                   labels={'category':'Kategori', 'count':'Jumlah'},
                   title='Distribusi Label di Validation Set',
                   color='category')

fig_valid.show()

# Visualisasi distribusi label untuk test set
fig_test = px.bar(test_counts,
                   x='category', y='count',
                   labels={'category':'Kategori', 'count':'Jumlah'},
                   title='Distribusi Label di Test Set',
                   color='category')

fig_test.show()


###  Menghitung persentase setiap label dalam dataset

In [167]:
df_train['category'].value_counts(normalize=True) * 100




category
positive    58.327273
negative    31.236364
neutral     10.436364
Name: proportion, dtype: float64

In [168]:
import plotly.graph_objs as go
import pandas as pd

# Hitung persentase setiap kategori
category_counts = df_train['category'].value_counts(normalize=True) * 100

# Buat pie chart 3D
fig = go.Figure(data=[go.Pie(
    labels=category_counts.index, 
    values=category_counts.values, 
    hole=0.3, 
    textinfo='percent+label',
    pull=[0.1, 0, 0]  # Untuk menonjolkan segmen tertentu jika diinginkan
)])

fig.update_layout(title_text='Category Distribution')


fig.show()

In [169]:
df_valid['category'].value_counts(normalize=True) * 100

category
positive    58.333333
negative    31.269841
neutral     10.396825
Name: proportion, dtype: float64

In [170]:
import plotly.graph_objs as go
import pandas as pd

# Hitung persentase setiap kategori
category_counts = df_valid['category'].value_counts(normalize=True) * 100

# Buat pie chart 3D
fig = go.Figure(data=[go.Pie(
    labels=category_counts.index, 
    values=category_counts.values, 
    hole=0.3, 
    textinfo='percent+label',
    pull=[0.1, 0, 0]  # Untuk menonjolkan segmen tertentu jika diinginkan
)])

fig.update_layout(title_text='Category Distribution')


fig.show()

In [171]:
df_test['category'].value_counts(normalize=True) * 100

category
positive    41.6
negative    40.8
neutral     17.6
Name: proportion, dtype: float64

In [172]:
import plotly.graph_objs as go
import pandas as pd

# Hitung persentase setiap kategori
category_counts = df_test['category'].value_counts(normalize=True) * 100

# Buat pie chart 3D
fig = go.Figure(data=[go.Pie(
    labels=category_counts.index, 
    values=category_counts.values, 
    hole=0.3, 
    textinfo='percent+label',
    pull=[0.1, 0, 0]  # Untuk menonjolkan segmen tertentu jika diinginkan
)])

fig.update_layout(title_text='Category Distribution')


fig.show()

# Pre-processing Dataset

### Case Folding

#### Langkah ini bertujuan untuk `mengonversi semua karakter menjadi huruf kecil`. Proses ini dilakukan karena data yang diperoleh sering kali tidak terstruktur dan tidak konsisten dalam penggunaan huruf kapital. Oleh karena itu, case folding dilakukan untuk `menyeragamkan penggunaan huruf kapital`.


In [173]:
# Mengonversi semua teks dalam kolom 'review_text' menjadi huruf kecil
df_train['review_text'] = df_train['review_text'].str.lower()
df_valid['review_text'] = df_valid['review_text'].str.lower()
df_test['review_text'] = df_test['review_text'].str.lower()

In [174]:
df_train

,review_text,category
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [175]:
df_test

,review_text,category
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative
...,...,...
495,kata nya tidur yang baik itu minimal enam jam ...,neutral
496,indonesia itu ada di benua asia .,neutral
497,salah satu kegemaran anak remaja indonesia sek...,neutral
498,melihat warna hijau bisa bikin mata jadi lebih...,positive


In [176]:
df_valid

,review_text,category
0,"meski masa kampanye sudah selesai , bukan bera...",neutral
1,tidak enak,negative
2,restoran ini menawarkan makanan sunda . kami m...,positive
3,lokasi di alun alun masakan padang ini cukup t...,positive
4,betapa bejad kader gerindra yang anggota dprd ...,negative
...,...,...
1255,"film tncfu , tidak cocok untuk penonton yang t...",negative
1256,"indihome ini mahal loh bayar nya . hanya , pen...",negative
1257,"be de gea , cowok cupu yang takut dengan pacar...",negative
1258,valen yang sangat tidak berkualitas . konentat...,negative


### Data Cleaning

#### Tahapan ini digunakan untuk `menghilangkan angka, beberapa simbol, url, username (@username), hashtag (#), spasi berlebih, tanda baca, emoji, dan pengulangan karakter yang ada pada kalimat`. Tahapan ini menggunakan `regular expression` untuk menemukan karakter yang akan dihapus.


In [177]:
import re

def clean_text(text):
    # Menghilangkan URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Menghilangkan username
    text = re.sub(r'\@\w+|\#', '', text)
    # Menghilangkan angka
    text = re.sub(r'\d+', '', text)
    # Menghilangkan tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    # Menghilangkan emoji
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Menghilangkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    # Menghilangkan pengulangan karakter
    text = re.sub(r'(.)\1+', r'\1\1', text)
    return text

# Menerapkan pembersihan teks pada kolom 'review_text'
df_train['review_text'] = df_train['review_text'].apply(clean_text)
df_valid['review_text'] = df_valid['review_text'].apply(clean_text)
df_test['review_text'] = df_test['review_text'].apply(clean_text)


In [178]:
df_train['category'].value_counts()


category
positive    6416
negative    3436
neutral     1148
Name: count, dtype: int64

In [179]:
# Mengonversi label menjadi angka
label_dict = {'negative': 0, 'positive': 1, 'neural' :2}
df_train['category'] = df_train['category'].map(label_dict)
df_valid['category'] = df_valid['category'].map(label_dict)
df_test['category'] = df_test['category'].map(label_dict)

### Tokenisasi

#### Tahapan ini menggunakan regular expression untuk menemukan karakter yang akan dihapus. Tahapan ini digunakan untuk memecah kalimat menjadi list kata. Proses ini menggunakan fungsi `word_tokenize` yang disediakan oleh library NLTK.


###  Instal dan Impor NLTK

In [180]:
!pip install nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [181]:
# Tokenisasi teks menggunakan word_tokenize dari NLTK
df_train['review_tokens'] = df_train['review_text'].apply(word_tokenize)
df_valid['review_tokens'] = df_valid['review_text'].apply(word_tokenize)
df_test['review_tokens'] = df_test['review_text'].apply(word_tokenize)


In [182]:
df_train

,review_text,category,review_tokens
0,warung ini dimiliki oleh pengusaha pabrik tahu...,1.0,"[warung, ini, dimiliki, oleh, pengusaha, pabri..."
1,mohon ulama lurus dan k mmbri hujjah partai ap...,NaN,"[mohon, ulama, lurus, dan, k, mmbri, hujjah, p..."
2,lokasi strategis di jalan sumatera bandung tem...,1.0,"[lokasi, strategis, di, jalan, sumatera, bandu..."
3,betapa bahagia nya diri ini saat unboxing pake...,1.0,"[betapa, bahagia, nya, diri, ini, saat, unboxi..."
4,duh jadi mahasiswa jangan sombong dong kasih k...,0.0,"[duh, jadi, mahasiswa, jangan, sombong, dong, ..."
...,...,...,...
10995,tidak kecewa,1.0,"[tidak, kecewa]"
10996,enak rasa masakan nya apalagi kepiting yang me...,1.0,"[enak, rasa, masakan, nya, apalagi, kepiting, ..."
10997,hormati partaipartai yang telah berkoalisi,NaN,"[hormati, partaipartai, yang, telah, berkoalisi]"
10998,pagi pagi di tol pasteur sudah macet parah bik...,0.0,"[pagi, pagi, di, tol, pasteur, sudah, macet, p..."


In [183]:
df_test

,review_text,category,review_tokens
0,kemarin gue datang ke tempat makan baru yang a...,0.0,"[kemarin, gue, datang, ke, tempat, makan, baru..."
1,kayak nya sih gue tidak akan mau balik lagi ke...,0.0,"[kayak, nya, sih, gue, tidak, akan, mau, balik..."
2,kalau dipikirpikir sebenarnya tidak ada yang b...,0.0,"[kalau, dipikirpikir, sebenarnya, tidak, ada, ..."
3,ini pertama kalinya gua ke bank buat ngurusin ...,0.0,"[ini, pertama, kalinya, gua, ke, bank, buat, n..."
4,waktu sampai dengan gue pernah disuruh ibu lat...,0.0,"[waktu, sampai, dengan, gue, pernah, disuruh, ..."
...,...,...,...
495,kata nya tidur yang baik itu minimal enam jam ...,NaN,"[kata, nya, tidur, yang, baik, itu, minimal, e..."
496,indonesia itu ada di benua asia,NaN,"[indonesia, itu, ada, di, benua, asia]"
497,salah satu kegemaran anak remaja indonesia sek...,NaN,"[salah, satu, kegemaran, anak, remaja, indones..."
498,melihat warna hijau bisa bikin mata jadi lebih...,1.0,"[melihat, warna, hijau, bisa, bikin, mata, jad..."


In [184]:
df_valid

,review_text,category,review_tokens
0,meski masa kampanye sudah selesai bukan berati...,NaN,"[meski, masa, kampanye, sudah, selesai, bukan,..."
1,tidak enak,0.0,"[tidak, enak]"
2,restoran ini menawarkan makanan sunda kami mem...,1.0,"[restoran, ini, menawarkan, makanan, sunda, ka..."
3,lokasi di alun alun masakan padang ini cukup t...,1.0,"[lokasi, di, alun, alun, masakan, padang, ini,..."
4,betapa bejad kader gerindra yang anggota dprd ...,0.0,"[betapa, bejad, kader, gerindra, yang, anggota..."
...,...,...,...
1255,film tncfu tidak cocok untuk penonton yang tid...,0.0,"[film, tncfu, tidak, cocok, untuk, penonton, y..."
1256,indihome ini mahal loh bayar nya hanya penanga...,0.0,"[indihome, ini, mahal, loh, bayar, nya, hanya,..."
1257,be de gea cowok cupu yang takut dengan pacar n...,0.0,"[be, de, gea, cowok, cupu, yang, takut, dengan..."
1258,valen yang sangat tidak berkualitas konentator...,0.0,"[valen, yang, sangat, tidak, berkualitas, kone..."


### Normalisasi Kata Tidak Baku

#### Tahap normalisasi adalah tahap di mana dataset yang `memiliki kata-kata tidak baku diubah menjadi kata yang baku atau sesuai dengan ejaan`. Hal ini dilakukan karena cukup banyak kalimat yang menggunakan `kata gaul seperti: tdk, dmn, cpt, ga, enggak, ngga, gak`. Jika kata tersebut tidak melalui tahap normalisasi, maka sistem akan menganggap kata ga, enggak, ngga, dan gak adalah kata yang berbeda, padahal kata tersebut memiliki makna yang seharusnya sama, yaitu enggak.


In [185]:
def normalize_text(text):
    # Daftar kata tidak baku dan kata yang sesuai
    replacements = {
        'tdk': 'tidak',
        'dmn': 'dimana',
        'cpt': 'cepat',
        'ga': 'tidak',
        'enggak': 'tidak',
        'ngga': 'tidak',
        'gak': 'tidak',
        'bgt': 'banget',
        'bgd': 'banget',
        'bnyk': 'banyak',
        'bgtu': 'begitu',
        'dgn': 'dengan',
        'hrs': 'harus',
        'knp': 'kenapa',
        'lgsg': 'langsung',
        'ngerti': 'mengerti',
        'pake': 'pakai',
        'sangat2': 'sangat',
        'sukak': 'suka',
        'syg': 'sayang',
        'ttg': 'tentang',
        'utk': 'untuk',
        'wkt': 'waktu',
        'yaa': 'ya',
        'bgitu': 'begitu',
        'ak': 'aku',
        'kau': 'kamu',
        'saya': 'aku',
        'anda': 'kamu',
        'kami': 'kita',
        'kalian': 'kamu',
        'dirimu': 'kamu',
        'dirinya': 'dia',
        'diriku': 'aku'
    }
    # Normalisasi kata tidak baku
    for key, value in replacements.items():
        text = text.replace(key, value)
    return text

# Normalisasi kata tidak baku pada kolom 'review_text'
df_train['review_text'] = df_train['review_text'].apply(normalize_text)
df_valid['review_text'] = df_valid['review_text'].apply(normalize_text)
df_test['review_text'] = df_test['review_text'].apply(normalize_text)


# Implementasi BERT

####  Pada penelitian ini, penulis menggunakan teknik fine-tuning dengan model `IndoBERT-base-p1`, salah satu model yang menggunakan `arsitektur BERT-base`. Teknik ini menggunakan model yang telah dilatih sebelumnya dan hanya belajar sedikit lagi untuk mencapai titik optimal pada task yang baru. Model ini telah dilatih menggunakan 4 miliar kata dengan sekitar 250 juta kalimat formal dan sehari-hari dalam Bahasa Indonesia (Wilie et al., 2020)

### Data Preparation
#### `BertTokenizer`, sebuah tokenizer yang bertujuan untuk melakukan tokenisasi pada kalimat-kalimat dan menghasilkan input yang sesuai. Hal ini dilakukan karena BERT menggunakan vocabulary yang spesifik yang mana tergantung dengan model apa yang dipakai.

In [140]:
!pip install transformers


#### Tokenisasi Dataset menggunakan BertTokenizer

In [141]:
# Inisialisasi tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenisasi dan padding
def tokenize_and_pad(sentences, max_length=64):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
            sent,                      # Kalimat yang akan ditokenisasi
            add_special_tokens=True,   # Tambahkan '[CLS]' dan '[SEP]'
            max_length=max_length,     # Padding & truncation length
            padding='max_length',      # Pad ke max_length
            return_attention_mask=True, # Return attention mask
            return_tensors='pt',       # Return pytorch tensors
            truncation=True            # Aktifkan truncation
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Konversi lists ke tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    return input_ids, attention_masks

# Fungsi untuk menyiapkan data
def prepare_data(sentences, labels, max_length=64, batch_size=32):
    input_ids, attention_masks = tokenize_and_pad(sentences, max_length=max_length)
    labels = torch.tensor(labels, dtype=torch.long)  # Pastikan labels adalah tipe long
    
    dataset = TensorDataset(input_ids, attention_masks, labels)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# Contoh penggunaan untuk dataset training
train_sentences = df_train['review_text'].values
train_labels = df_train['category'].values
train_dataloader = prepare_data(train_sentences, train_labels)

# Contoh penggunaan untuk dataset validasi
valid_sentences = df_valid['review_text'].values
valid_labels = df_valid['category'].values
valid_dataloader = prepare_data(valid_sentences, valid_labels)

# Contoh penggunaan untuk dataset test
test_sentences = df_test['review_text'].values
test_labels = df_test['category'].values
test_dataloader = prepare_data(test_sentences, test_labels)


# Model Training

### Inisialisasi Common Functions

#### Berikut adalah inisialisasi common function yang digunakan sebagai persiapan dalam menggunakan model BERT sekaligus BertTokenize

In [142]:
pip install torch torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.


In [143]:
pip install --upgrade pip setuptools


Note: you may need to restart the kernel to use updated packages.


In [144]:
pip install torch


In [145]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [146]:
import torch
from transformers import BertTokenizer

# Cek apakah PyTorch terinstal dengan benar
print(torch.__version__)
print(BertTokenizer)


2.3.1+cpu
<class 'transformers.models.bert.tokenization_bert.BertTokenizer'>


In [147]:
from transformers import BertTokenizer

# Inisialisasi tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Contoh kalimat
sentence = "Ini adalah contoh kalimat untuk tokenisasi."

# Tokenisasi kalimat menggunakan BERT tokenizer
tokens = tokenizer.tokenize(sentence)
print(f"Tokens: {tokens}")

# Konversi token menjadi indeks
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
print(f"Indexed Tokens: {indexed_tokens}")

# Konversi indeks menjadi tensor PyTorch
import torch
tensor = torch.tensor([indexed_tokens])
print(f"Tensor: {tensor}")


Tokens: ['in', '##i', 'ada', '##lah', 'con', '##to', '##h', 'kali', '##mat', 'un', '##tu', '##k', 'token', '##isa', '##si', '.']
Indexed Tokens: [1999, 2072, 15262, 14431, 9530, 3406, 2232, 19924, 18900, 4895, 8525, 2243, 19204, 14268, 5332, 1012]
Tensor: tensor([[ 1999,  2072, 15262, 14431,  9530,  3406,  2232, 19924, 18900,  4895,
          8525,  2243, 19204, 14268,  5332,  1012]])


In [148]:
import torch
print(f"PyTorch version: {torch.__version__}")


PyTorch version: 2.3.1+cpu


In [149]:
import torch
from transformers import BertTokenizer

# Cek versi PyTorch
print(f"PyTorch version: {torch.__version__}")

# Inisialisasi tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Contoh kalimat
sentences = ["Ini adalah kalimat contoh untuk pengujian tokenisasi BERT."]

# Tokenisasi dan padding
def tokenize_and_pad(sentences, max_length=64):
    input_ids = []
    attention_masks = []

    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
            sent,                      # Kalimat yang akan ditokenisasi
            add_special_tokens=True,   # Tambahkan '[CLS]' dan '[SEP]'
            max_length=max_length,     # Padding & truncation length
            padding='max_length',      # Pad ke max_length
            return_attention_mask=True, # Return attention mask
            return_tensors='pt',       # Return pytorch tensors
            truncation=True            # Aktifkan truncation
        )
        
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Konversi lists ke tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    return input_ids, attention_masks

input_ids, attention_masks = tokenize_and_pad(sentences)

print("Input IDs:", input_ids)
print("Attention Masks:", attention_masks)


PyTorch version: 2.3.1+cpu
Input IDs: tensor([[  101,  1999,  2072, 15262, 14431, 19924, 18900,  9530,  3406,  2232,
          4895,  8525,  2243, 26473, 23049,  2937, 19204, 14268,  5332, 14324,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])
Attention Masks: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [150]:
import random
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
import time
import datetime

# Inisialisasi common functions
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

def count_param(module, trainable=False):
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def metrics_to_string(metric_dict):
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

# Set random seed for reproducibility
set_seed(27)


# Model Initialization

In [151]:
# Inisialisasi model BERT untuk sequence classification
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2,  # Misalkan kita punya 2 label
    output_attentions=False,
    output_hidden_states=False,
)

# Gunakan GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Training and Evaluation Functions

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:588: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning



Epoch 1/4


IndexError: Target -9223372036854775808 is out of bounds.

#### Inisialisasi Kelas DocumentSentimentDataset dan DocumentSentimentDataLoader

#### Kelas DocumentSentimentDataset
Kelas ini akan menangani penyimpanan dan pengolahan data teks dan label.

In [223]:
from torch.utils.data import Dataset

class DocumentSentimentDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_length):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]
        
        # Tokenisasi dan padding
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


####  Kelas DocumentSentimentDataLoader
Kelas ini akan menangani pembuatan DataLoader untuk training, validation, dan test set.

In [224]:
from torch.utils.data import DataLoader

class DocumentSentimentDataLoader:
    def __init__(self, dataset, batch_size, sampler):
        self.dataset = dataset
        self.batch_size = batch_size
        self.sampler = sampler
    
    def get_dataloader(self):
        return DataLoader(
            self.dataset,
            sampler=self.sampler(self.dataset),
            batch_size=self.batch_size
        )


#### Contoh Penggunaan
Sekarang kita bisa menggunakan kedua kelas ini untuk mempersiapkan data dan dataloader

In [225]:
# Inisialisasi tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Data training
train_sentences = df_train['review_text'].values
train_labels = df_train['category'].values

# Data validation
valid_sentences = df_valid['review_text'].values
valid_labels = df_valid['category'].values

# Data test
test_sentences = df_test['review_text'].values
test_labels = df_test['category'].values

# Inisialisasi dataset
train_dataset = DocumentSentimentDataset(train_sentences, train_labels, tokenizer, max_length=64)
valid_dataset = DocumentSentimentDataset(valid_sentences, valid_labels, tokenizer, max_length=64)
test_dataset = DocumentSentimentDataset(test_sentences, test_labels, tokenizer, max_length=64)

# Inisialisasi dataloader
train_dataloader_obj = DocumentSentimentDataLoader(train_dataset, batch_size=32, sampler=RandomSampler)
valid_dataloader_obj = DocumentSentimentDataLoader(valid_dataset, batch_size=32, sampler=SequentialSampler)
test_dataloader_obj = DocumentSentimentDataLoader(test_dataset, batch_size=32, sampler=SequentialSampler)

train_dataloader = train_dataloader_obj.get_dataloader()
valid_dataloader = valid_dataloader_obj.get_dataloader()
test_dataloader = test_dataloader_obj.get_dataloader()


#### Fase Training dan Evaluasi

Training tidak dilakukan dari awal, melainkan dengan menggunakan model yang telah dilatih sebelumnya dan hanya belajar sedikit lagi untuk mencapai titik optimal pada task yang baru. Teknik training ini disebut sebagai fine-tuning. Dengan cara ini, tidak perlu melakukan training dari awal, tetapi kita dapat mendownload model yang telah dilatih sebelumnya (pre-trained model).

#### Mengunduh dan Memuat Model Pre-trained BERT
Kita akan menggunakan model BertForSequenceClassification dari Hugging Face Transformers.

In [226]:
from transformers import BertForSequenceClassification, AdamW

# Load pre-trained model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',  # Nama pre-trained model
    num_labels=2,         # Jumlah label untuk classification
    output_attentions=False,
    output_hidden_states=False
)

# Pindahkan model ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

#### Fine-tuning
Kita akan menggunakan optimizer AdamW dan scheduler learning rate untuk fine-tuning model.

#### Persiapan Dataloader

In [227]:
#### Contoh data
sentences = ["Ini adalah kalimat contoh untuk pengujian tokenisasi BERT."]
labels = [1]  # Label kategori

input_ids, attention_masks = tokenize_and_pad(sentences)
labels = torch.tensor(labels)

# Membuat TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Membuat DataLoader
batch_size = 2

# Dataloader untuk training set
train_dataloader = DataLoader(
    dataset,
    sampler=RandomSampler(dataset),
    batch_size=batch_size
)

# Dataloader untuk validation set (contoh, gunakan data validasi sebenarnya)
validation_dataloader = DataLoader(
    dataset,
    sampler=SequentialSampler(dataset),
    batch_size=batch_size
)


#### Fungsi untuk Format Waktu

In [228]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round(elapsed))))


#### Pelatihan Model 

In [229]:
# Set jumlah epochs
epochs = 2

# Loop pelatihan
for epoch_i in range(0, epochs):
    print(f'Epoch {epoch_i + 1}/{epochs}')
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_attention_mask, b_labels = batch
        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'Batch {step} of {len(train_dataloader)}. Loss: {loss.item()}. Elapsed: {elapsed}.')

    avg_train_loss = total_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print(f'Average training loss: {avg_train_loss}')
    print(f'Training epoch took: {training_time}')

    # Evaluasi pada validation set
    print('Running Validation...')

    t0 = time.time()
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        b_input_ids, b_attention_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_attention_mask)

        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        eval_accuracy += np.sum(np.argmax(logits, axis=1) == label_ids)
        nb_eval_steps += 1

    print(f'Validation Accuracy: {eval_accuracy/nb_eval_steps}')
    print(f'Validation took: {format_time(time.time() - t0)}')


Epoch 1/2
Training...
Average training loss: 0.8998916149139404
Training epoch took: 0:00:01
Running Validation...
Validation Accuracy: 0.0
Validation took: 0:00:00
Epoch 2/2
Training...
Average training loss: 1.0139402151107788
Training epoch took: 0:00:01
Running Validation...
Validation Accuracy: 0.0
Validation took: 0:00:01
